# Netflix Text Analytics

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, TypeAlias, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Load Data

In [2]:
from pathlib import Path
import pyarrow.dataset as ds

# Specify the directory containing Parquet files
directory_path: Path = Path("../../../../data/NETFLIX_REVIEWS")

# Use PyArrow to read the partitioned Parquet dataset
table = ds.dataset(directory_path)

# Convert the PyArrow table to a Pandas DataFrame
data: pl.DataFrame = pl.scan_pyarrow_dataset(table).collect()

# Display the DataFrame
print(data)

shape: (1_531_126, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ review_id    ┆ pseudo_auth ┆ author_name ┆ review_text ┆ review_like ┆ author_app_ ┆ review_time │
│ ---          ┆ or_id       ┆ ---         ┆ ---         ┆ s           ┆ version     ┆ stamp       │
│ str          ┆ ---         ┆ str         ┆ str         ┆ ---         ┆ ---         ┆ ---         │
│              ┆ str         ┆             ┆             ┆ i64         ┆ str         ┆ str         │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 5021e061-326 ┆ 22072027565 ┆ A Google    ┆ The app     ┆ 0           ┆ 1.5.2 build ┆ 2011-05-19  │
│ a-4a57-892e- ┆ 0827922209  ┆ user        ┆ would be    ┆             ┆ 389         ┆ 02:49:49    │
│ 7635b3e0…    ┆             ┆             ┆ good if it  ┆             ┆             ┆             │
│              ┆             ┆             ┆ can …       ┆             ┆             ┆             │
│ a2f3fcc6-0c2 ┆ 19073660296 ┆ A Google    ┆ Does not    ┆ 0           ┆ 1.2.1 build ┆ 2011-06-06  │
│ 0-41ee-bbd6- ┆ 1554344592  ┆ user        ┆ work w      ┆             ┆ 843839-1.2. ┆ 00:53:58    │
│ 60670c8a…    ┆             ┆             ┆ Dreambook   ┆             ┆ 0-30        ┆             │
│              ┆             ┆             ┆ W7 run…     ┆             ┆             ┆             │
│ 7f9ab522-f18 ┆ 16251035459 ┆ A Google    ┆ Plays video ┆ 0           ┆ 1.5.1 build ┆ 2011-06-23  │
│ 7-4e81-bbfc- ┆ 5177818389  ┆ user        ┆ for like    ┆             ┆ 367         ┆ 19:44:33    │
│ 3c3cddfd…    ┆             ┆             ┆ five mins   ┆             ┆             ┆             │
│              ┆             ┆             ┆ t…          ┆             ┆             ┆             │
│ 78cf6a08-f2c ┆ 96280635786 ┆ A Google    ┆ It works    ┆ 0           ┆ 1.5.1 build ┆ 2011-06-27  │
│ e-41d2-b003- ┆ 2315221677  ┆ user        ┆ great on    ┆             ┆ 367         ┆ 21:19:04    │
│ 68cd5cbe…    ┆             ┆             ┆ Droid X ,   ┆             ┆             ┆             │
│              ┆             ┆             ┆ work…       ┆             ┆             ┆             │
│ …            ┆ …           ┆ …           ┆ …           ┆ …           ┆ …           ┆ …           │
│ ccfafe2a-37e ┆ 34687258596 ┆ Kh********* ┆ I love the  ┆ 0           ┆ 8.94.0      ┆ 2023-11-15  │
│ 9-4717-b0b1- ┆ 0277545023  ┆ **ko        ┆ way it was  ┆             ┆ build 10    ┆ 20:39:04    │
│ 1ea565e4…    ┆             ┆             ┆ working     ┆             ┆ 50546       ┆             │
│ 00cb5f63-81c ┆ 33735617720 ┆ Tr********* ┆ I love      ┆ 0           ┆             ┆ 2023-11-15  │
│ a-4ffc-ab9f- ┆ 8708791652  ┆ ****ek      ┆ total drama ┆             ┆             ┆ 21:46:34    │
│ f2f1259b…    ┆             ┆             ┆             ┆             ┆             ┆             │
│ 50a374b0-221 ┆ 84765653469 ┆ Ed********* ┆ I love my   ┆ 0           ┆ 8.26.0      ┆ 2023-11-15  │
│ c-41e8-b853- ┆ 4907718280  ┆ *la         ┆ Netflix     ┆             ┆ build 11    ┆ 21:58:00    │
│ 5c26b961…    ┆             ┆             ┆             ┆             ┆ 40221       ┆             │
│ 3015ab73-75e ┆ 26838594181 ┆ Em********* ┆ I will love ┆ 0           ┆             ┆ 2023-11-15  │
│ 8-4f17-8377- ┆ 1343301666  ┆ ***ey       ┆ this app    ┆             ┆             ┆ 22:45:05    │
│ 4757abbb…    ┆             ┆             ┆             ┆             ┆             ┆             │
└──────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┘